# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [55]:
# Details Student 1:Daniel Zatakovi 314915984 dani.zatak@gmail.com

# Details Student 2:Jeki Skif 318973898 jjeki5858@gmail.com


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [56]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2 ,mutual_info_classif

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [57]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [58]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [59]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [60]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [61]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [62]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [63]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [64]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [65]:
def Remove_UnnecessaryChars(str):
    str = re.sub(r'\d+', '', str)
    str = re.sub(r'\s+', ' ', str) 
    str = re.sub(r'[^\w\s]', '', str) 
    str = str.strip() 
    return str

In [66]:
def Create_Train_Data(df: pd.DataFrame):
    
    df['story'] = df['story'].apply(Remove_UnnecessaryChars)
    vectorTxt_Train = TfidfVectorizer(ngram_range = (1,1), min_df = 5)
    
    X_train, X_test, y_train, y_test = train_test_split(df['story'], df['gender'], test_size=0.2, random_state=42)
    X_train_vec = vectorTxt_Train.fit_transform(X_train)
    X_test_vec = vectorTxt_Train.transform(X_test)

    selection = SelectKBest(mutual_info_classif, k=1000)
    selection.fit(X_train_vec, y_train)

    X_train_selection = selection.transform(X_train_vec)
    X_test_selection = selection.transform(X_test_vec)

    scale_features= MinMaxScaler()
    X_train = scale_features.fit_transform(X_train_selection.toarray())
    X_test = scale_features.transform(X_test_selection.toarray())
    
    return X_train, X_test, y_train, y_test

In [67]:
def Create_Test_Data(df: pd.DataFrame, y_train: np.array):

     df['story'] = df['story'].apply(Remove_UnnecessaryChars)
     vectorTxt_Test = TfidfVectorizer(ngram_range = (1,1), min_df = 5)

     X_test = df['story']
     X_test_vec = vectorTxt_Test.fit_transform(X_test)
    
     selection = SelectKBest(mutual_info_classif, k=1000) 
     X_test_selection = selection.fit_transform(X_test_vec, np.zeros(X_test_vec.shape[0]))

     scale_features = MinMaxScaler()
     X_test = scale_features.fit_transform(X_test_selection.toarray())

     return df['story'], X_test

In [68]:
def Best_LogisticRegression_Model(X_train: np.array, y_train: np.array):

    logistic_reg = linear_model.LogisticRegression()
    logistic_reg_params = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}

    grid_search = GridSearchCV(logistic_reg, logistic_reg_params, cv=5, n_jobs = -1)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    
    lr_best_model = linear_model.LogisticRegression(**best_params)
    lr_best_model.fit(X_train, y_train)  

    return lr_best_model


In [69]:
def Best_NaiveBayes_Model(X_train: np.array, y_train: np.array):
 
    naive_bayes = MultinomialNB()
    naive_bayes_params = {'alpha': [0.1, 1, 10], 'fit_prior': [True, False], 'class_prior': [None, [0.3, 0.7], [0.5, 0.5]]}

    # Find best params using GridSearchSV
    grid_search = GridSearchCV(naive_bayes, naive_bayes_params, cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_

    # Create New Model based on best params and train it with our data
    nb_best_model = MultinomialNB(**best_params)
    nb_best_model.fit(X_train, y_train) 

    return nb_best_model

In [70]:
def Print_Result(model_name, y_test, y_pred, score):
    f1_male = f1_score(y_test, y_pred, pos_label='m')
    f1_female = f1_score(y_test, y_pred, pos_label='f')
    f1_average =(f1_male + f1_female)/2

    print('Model : ', model_name)
    print('Results:')
    print("---------")
    print('f1_Male : %.4f' % f1_male)
    print('f1_Female : %.4f' % f1_female)
    print('f1_Average: %.4f \n' % f1_average)
    print('Accuracy : %.4f | std: %.4f' % (np.mean(score),np.std(score)))

In [71]:
def Print_Predictions(model_name, y_test, y_pred):
    
    df_predicted = pd.DataFrame({'story': y_test, 'predicted gender': y_pred})
    
    print('Model Prediction: ' , model_name)
    print('Top5')
    print(df_predicted.head(5))
    print('Last5')
    print(df_predicted.tail(5))
    print('Total')
    print(df_predicted['predicted gender'].value_counts())

In [72]:
X_train, X_test, y_train, y_test = Create_Train_Data(df_train)

logistic_reg_model = Best_LogisticRegression_Model(X_train, y_train)
naive_bayes_model = Best_NaiveBayes_Model(X_train, y_train)

cv = model_selection.RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
logistic_reg_score = cross_val_score(logistic_reg_model, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1)
naive_bayes_score = cross_val_score(naive_bayes_model, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1)

logistic_reg_y_predict = logistic_reg_model.predict(X_test)
naive_bayes_y_predict = naive_bayes_model.predict(X_test)

Print_Result('Logistic Regression', y_test, logistic_reg_y_predict, logistic_reg_score)
Print_Result('Naive Bayes', y_test, naive_bayes_y_predict, naive_bayes_score)


test_Data, test_X_test = Create_Test_Data(df_test, y_train)
logistic_reg_test_y_predict = logistic_reg_model.predict(test_X_test)
naive_bayes_test_y_pred = naive_bayes_model.predict(test_X_test)

Print_Predictions('Logistic Regression', test_Data, logistic_reg_test_y_predict)
Print_Predictions('Naive Bayes', test_Data, naive_bayes_test_y_pred)

Model :  Logistic Regression
Results:
---------
f1_Male : 0.8790
f1_Female : 0.4444
f1_Average: 0.6617 

Accuracy : 0.7593 | std: 0.0945
Model :  Naive Bayes
Results:
---------
f1_Male : 0.8816
f1_Female : 0.4912
f1_Average: 0.6864 

Accuracy : 0.7482 | std: 0.0878
Model Prediction:  Logistic Regression
Top5
                                               story predicted gender
0  כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...                m
1  הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת י...                m
2  אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...                m
3  רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי אני ...                f
4  אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...                m
Last5
                                                 story predicted gender
318  בשנה האחרונה הרגשתי די תקוע בעבודה השגרה הפכה ...                m
319  אני ואילן חברים טובים מזה שנה תמיד חלמנו לפתח ...                f
320  מידי יום שישי אני נוהג לנסוע בתחבורה ציבורי

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [74]:
df_predicted.to_csv('classification_results.csv',index=False)

NameError: name 'df_predicted' is not defined